In [1]:
from os import listdir

import time
import json
import copy

import torch

from torchvision.datasets import mnist, FashionMNIST, CIFAR10, CIFAR100
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.optim import SGD, Adam
from torch.nn import Module
from torch import nn
from torch.nn import CrossEntropyLoss
from torchvision.models.resnet import Bottleneck, ResNet
from torchvision import datasets, models, transforms

from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

from openood.evaluators import metrics

In [2]:
print(torch.version.cuda)

10.2


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
!pwd

/home/rdr2143/oodn-final-project/OpenOOD-nndl


### Supported Activation Functions

For activation functions, we are considering ReLU, Softplus, Swish. *Note that we may conduct experiments for a subset based on the compute resources available*

In [5]:
def get_activation_fn(activation):
    if activation == 'relu':
        return nn.ReLU()
    elif activation == 'softplus':
        return nn.Softplus()
    elif activation == 'swish':
        return nn.Swish()
    return None

### LeNet

In [6]:
class LeNet(nn.Module):
    def __init__(self, num_classes, num_channel=3, activation='relu'):
        super(LeNet, self).__init__()
        self.num_classes = num_classes
        self.feature_size = 84
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=num_channel,
                      out_channels=6,
                      kernel_size=5,
                      stride=1,
                      padding=2), get_activation_fn(activation), nn.MaxPool2d(kernel_size=2))

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
             get_activation_fn(activation), nn.MaxPool2d(kernel_size=2))

        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=16,
                      out_channels=120,
                      kernel_size=5,
                      stride=1), get_activation_fn(activation))

        self.classifier1 = nn.Linear(in_features=120, out_features=84)
        self.relu = get_activation_fn(activation)
        self.fc = nn.Linear(in_features=84, out_features=num_classes)

    def get_fc(self):
        fc = self.fc
        return fc.weight.cpu().detach().numpy(), fc.bias.cpu().detach().numpy()

    def forward(self, x, return_feature=False, return_feature_list=False):
        feature1 = self.block1(x)
        feature2 = self.block2(feature1)
        feature3 = self.block3(feature2)
        feature3 = feature3.view(feature3.shape[0], -1)
        feature = self.relu(self.classifier1(feature3))
        logits_cls = self.fc(feature)
        feature_list = [feature1, feature2, feature3, feature]
        if return_feature:
            return logits_cls, feature
        elif return_feature_list:
            return logits_cls, feature_list
        else:
            return logits_cls

    def forward_threshold(self, x, threshold):
        feature1 = self.block1(x)
        feature2 = self.block2(feature1)
        feature3 = self.block3(feature2)
        feature3 = feature3.view(feature3.shape[0], -1)
        feature = self.relu(self.classifier1(feature3))
        feature = feature.clip(max=threshold)
        logits_cls = self.fc(feature)

        return logits_cls

### ResNet50

In [7]:
def set_parameter_requires_grad(model):
    for name,param in model.named_parameters():
        if not (name.startswith('layer4') or name.startswith('fc')):
            param.requires_grad = False

def get_resnet_model(activation_function_type, n_classes, use_pretrained=True):
    resnet_model = models.resnet50(pretrained=use_pretrained)
    
    # if we use pretrained, then freeze the corresponding layers
    if use_pretrained:
        set_parameter_requires_grad(resnet_model, feature_extract)

    set_activation_function(resnet_model,activation_function_type)
    num_ftrs = resnet_model.fc.in_features
    resnet_model.fc = nn.Linear(num_ftrs, n_classes)
    resnet_model.to(device)
    return resnet_model

def set_activation_function(resnet_model, activation_function_type):
    resnet_model.relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[2].relu = get_activation_fn(activation_function_type)

    resnet_model.layer2[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[2].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[3].relu = get_activation_fn(activation_function_type)

    resnet_model.layer3[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[2].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[3].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[4].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[5].relu = get_activation_fn(activation_function_type)


    resnet_model.layer4[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer4[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer4[2].relu = get_activation_fn(activation_function_type)

    return resnet_model

In [8]:
def get_model(config):
    activation_function_type = config["activation_function_type"]
    network_type = config["network"]
    n_classes = config["n_classes"]

    if network_type == "lenet":
        model =  LeNet(num_classes=n_classes, num_channel=1, activation=activation_function_type)
    elif network_type == "resnet50":
        model = get_resnet_model(activation_function_type, n_classes, config['pretrained'])
    else:
        raise Exception("Currently we only support lenet or resnet50")

    return model

### Supported Post-Hoc OODN Processors

In [9]:
class OODPostprocessor():
    
    def inference(self, net: nn.Module, data_loader: DataLoader):
        pred_list, conf_list, label_list = [], [], []
        for idx, loaded_data in enumerate(data_loader):
            data, label = loaded_data[0], loaded_data[1]
            if idx % 50 == 0:
                print(f'Performing inference on batch: {idx}')
            pred, conf = self.postprocess(net, data.to(device))
            for idx in range(len(data)):
                pred_list.append(pred[idx].tolist())
                conf_list.append(conf[idx].tolist())
                label_list.append(label[idx].tolist())

        # convert values into numpy array
        pred_list = np.array(pred_list, dtype=int)
        conf_list = np.array(conf_list)
        label_list = np.array(label_list, dtype=int)

        return pred_list, conf_list, label_list

In [10]:
class ODINPostprocessor(OODPostprocessor):
    def __init__(self, temperature, noise):
        super(OODPostprocessor)
        self.temperature = temperature
        self.noise = noise
        
    def postprocess(self, net: nn.Module, data):
        net.eval()
        data.requires_grad = True
        output = net(data)

        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        criterion = nn.CrossEntropyLoss()

        labels = output.detach().argmax(axis=1)

        # Using temperature scaling
        output = output / self.temperature

        loss = criterion(output, labels)
        loss.backward()

        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(data.grad.detach(), 0)
        gradient = (gradient.float() - 0.5) * 2

        # Scaling values taken from original code       
        gradient[:, 0] = (gradient[:, 0]) / (63.0 / 255.0)
        if gradient.shape[1] == 3:
            gradient[:, 1] = (gradient[:, 1]) / (62.1 / 255.0)
            gradient[:, 2] = (gradient[:, 2]) / (66.7 / 255.0)

        # Adding small perturbations to images
        tempInputs = torch.add(data.detach(), gradient, alpha=-self.noise)
        output = net(tempInputs)
        output = output / self.temperature

        # Calculating the confidence after adding perturbations
        nnOutput = output.detach()
        nnOutput = nnOutput - nnOutput.max(dim=1, keepdims=True).values
        nnOutput = nnOutput.exp() / nnOutput.exp().sum(dim=1, keepdims=True)

        conf, pred = nnOutput.max(dim=1)

        return pred, conf

In [11]:
class MCDPostprocessor(OODPostprocessor):
    def __init__(self, samples: int = 30):
        super(OODPostprocessor)
        self.samples = samples  #: number :math:`N` of samples

    def postprocess(self, model: torch.nn.Module, x: torch.Tensor) -> torch.Tensor:
        mode_switch = False
        if not model.training:
            mode_switch = True

            model.train()

            for mod in model.modules():
                # reset batch norm layers.
                # TODO: are there other layers?
                if isinstance(mod, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)):
                    mod.train(False)

        results = None
        with torch.no_grad():
            for i in range(self.samples):
                output = model(x).softmax(dim=1)
                if results is None:
                    results = torch.zeros(size=output.shape).to(device)
                results += output
        results /= self.samples

        if mode_switch:
            model.eval()
        
        conf, pred = results.max(dim=1)

        return pred, conf

In [12]:
def get_postprocessor(postprocessor_type="odin"):
    if postprocessor_type == "odin":
        postprocessor = ODINPostprocessor(1000, 0.0014)
    elif postprocessor_type == "mcd":
        postprocessor = MCDPostprocessor(30)
    return postprocessor

### Supported Out of Distribution Detection Metrics

What metrics do we specifically care about here?

**FPR@95** measures the false positive rate (FPR) when the true positive rate (TPR) is
equal to 95%. Lower scores indicate better performance.

**AUROC** measures the area under the
Receiver Operating Characteristic (ROC) curve, which displays the relationship between TPR and
FPR. The area under the ROC curve can be interpreted as the probability that a positive ID example
will have a higher detection score than a negative OOD example.

**AUPR** measures the area under
the Precision-Recall (PR) curve. The PR curve is created by plotting precision versus recall. Similar
to AUROC, we consider ID samples as positive, so that the score corresponds to the AUPR-In metric
in some works

In [13]:
def calculate_oodn_metrics(model, postprocessor_type, id_test_loader, ood_test_loader, ood_name):
    postprocessor = get_postprocessor(postprocessor_type)
    id_pred, id_conf, id_gt = postprocessor.inference(
                model, id_test_loader)

    ood_pred, ood_conf, ood_gt = postprocessor.inference(
        model, ood_test_loader)

    ood_gt = -1 * np.ones_like(ood_gt)  # hard set to -1 as ood
    pred = np.concatenate([id_pred, ood_pred])
    conf = np.concatenate([id_conf, ood_conf])
    label = np.concatenate([id_gt, ood_gt])
    ood_metrics = metrics.compute_all_metrics(conf, label, pred)

    return print_and_get_formatted_metrics(ood_metrics, ood_name)

def print_and_get_formatted_metrics(metrics, dataset_name):
    [fpr, auroc, aupr_in, aupr_out,
     ccr_4, ccr_3, ccr_2, ccr_1, accuracy] \
     = metrics

    write_content = {
        'dataset': dataset_name,
        'FPR@95': '{:.2f}'.format(100 * fpr),
        'AUROC': '{:.2f}'.format(100 * auroc),
        'AUPR_IN': '{:.2f}'.format(100 * aupr_in),
        'AUPR_OUT': '{:.2f}'.format(100 * aupr_out),
        'CCR_4': '{:.2f}'.format(100 * ccr_4),
        'CCR_3': '{:.2f}'.format(100 * ccr_3),
        'CCR_2': '{:.2f}'.format(100 * ccr_2),
        'CCR_1': '{:.2f}'.format(100 * ccr_1),
        'ACC': '{:.2f}'.format(100 * accuracy)
    }

    fieldnames = list(write_content.keys())

    # print ood metric results
    print('FPR@95: {:.2f}, AUROC: {:.2f}'.format(100 * fpr, 100 * auroc),
          end=' ',
          flush=True)
    print('AUPR_IN: {:.2f}, AUPR_OUT: {:.2f}'.format(
        100 * aupr_in, 100 * aupr_out),
          flush=True)
    print('CCR: {:.2f}, {:.2f}, {:.2f}, {:.2f},'.format(
        ccr_4 * 100, ccr_3 * 100, ccr_2 * 100, ccr_1 * 100),
          end=' ',
          flush=True)
    print('ACC: {:.2f}'.format(accuracy * 100), flush=True)
    print(u'\u2500' * 70, flush=True)
    return write_content

def load_results_into_df(dir_path):
    res_files = [dir_path+each for each in listdir(dir_path)]
    all_results = []
    columns = ['optimizer_type', 'activation_function_type', 'postprocessor_type', 'trial', 'AUROC']
    for fp in res_files:
        f = open(fp)
        data = json.load(f)
        for trial, results in data.items():
            all_results.append([
                    results['optimizer_type'],
                    results['activation_function_type'],
                    results['postprocessor_type'],
                    trial,
                    float(results['AUROC'])
                ])
    df = pd.DataFrame(all_results, columns=columns)
    return df

In [21]:
def get_optimizer(model, config):
    params = model.parameters()
    lr = config['lr']
    momentum = config['momentum']
    weight_decay = config['weight_decay']
    optimizer_type = config['optimizer_type']

    print(f'Getting optimizer for type: {optimizer_type}...')
    if optimizer_type == 'SGD':
        return SGD(params,
              lr=lr,
              momentum=momentum,
              weight_decay=weight_decay)
    elif optimizer_type == 'Adam':
        return Adam(params,
                    lr=lr,
                    weight_decay=weight_decay)
    else:
        raise Exception("Invalid optimizer_type provided, only SGD and Adam are supported currently")

def get_wilds_loader(dataset, split, batch_size):
    d = dataset.get_subset(
        split,
        # frac=0.1,
        transform=transforms.Compose(
            [transforms.Resize((448, 448)), transforms.ToTensor()]
        ),
    )
    # Prepare the standard data loader
    return get_train_loader("standard", d, batch_size=batch_size, num_workers=4)

def get_data_loaders(config):
    data_loaders = {}
    dataset_name = config["dataset_name"]
    dataset_type = config["dataset_type"]
    batch_size = config['batch_size']

    wilds_id_test_split = "id_val" if dataset_name == "camelyon17" else "id_test"
    if dataset_type == "wilds":
        # wilds dataset
        dataset = get_dataset(dataset=dataset_name, download=True)
        data_loaders["train"] = get_wilds_loader(dataset, "train", batch_size)
        data_loaders["ood_test"] = get_wilds_loader(dataset, "test", batch_size)
        data_loaders["id_test"] = get_wilds_loader(dataset, wilds_id_test_split, batch_size)
        return
    elif dataset_name == "cifar":
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        train_dataset = CIFAR10(root='data', download=True, train=True, transform=transform_train)
        test_dataset = CIFAR10(root='data', download=True, train=False, transform=transform_test)
        ood_test_dataset = CIFAR100(root='data', download=True, train=False, transform=transform_test)
    elif dataset_name == "mnist":
        # mnist dataset
        train_dataset = mnist.MNIST(root='data', download=True, train=True, transform=ToTensor())
        test_dataset = mnist.MNIST(root='data', download=True, train=False, transform=ToTensor())
        ood_test_dataset = mnist.FashionMNIST(root='data', download=True,train=False,transform=ToTensor())

    data_loaders["train"] = DataLoader(train_dataset, batch_size=batch_size)
    data_loaders["id_test"] = DataLoader(test_dataset, batch_size=batch_size)
    data_loaders["ood_test"] = DataLoader(ood_test_dataset, batch_size=batch_size)

    return data_loaders

In [22]:
def train_resnet_model_given_opti_activation_fn(config):
    # get the train loader
    train_loader = config["data_loaders"]["train"]

    # get the resnet model with the replaced activation functions
    model = get_model(config)
    model.to(device)

    # get the optimizer
    sgd = get_optimizer(model, config)

    loss_fn = CrossEntropyLoss()
    for current_epoch in range(config['epochs']):
        tic=time.time()
        per_batch_time = time.time()
        model.train()
        print('Training epoch: {}'.format(current_epoch))
        for idx, (loader_data) in enumerate(train_loader):
            train_x, train_label = loader_data[0].to(device), loader_data[1].to(device)
            sgd.zero_grad()
            predict_y = model(train_x.float())
            loss = loss_fn(predict_y, train_label.long())
            if idx % 100 == 0:
                print('idx: {}, loss: {} time take: {}'.format(idx, loss.sum().item(), time.time() - per_batch_time))
                per_batch_time = time.time()
            loss.backward()
            sgd.step()
        print(f"epoch {current_epoch} time taken: {time.time()-tic}s")
    torch.save(model, config['model_name'])

    return model

def run_full_oodn_pipeline(config):
    metrics = {}
    for i in range(config["trials"]):
        model_name = f"models/{config['dataset_name']}_{config['network']}_{config['postprocessor_type']}_{config['activation_function_type']}_{config['optimizer_type']}_{i}.pkl"
        print(f'Running model: {model_name}...')
        config['model_name'] = model_name
        # train model
        model = train_resnet_model_given_opti_activation_fn(config)
        # calculate oodn metrics
        metrics[i] = calculate_oodn_metrics(model,
                               config['postprocessor_type'],
                               config["data_loaders"]["id_test"],
                               config["data_loaders"]["ood_test"],
                               config["dataset_name"])
        metrics[i]['optimizer_type'] = config['optimizer_type']
        metrics[i]['activation_function_type'] = config['activation_function_type']
        metrics[i]['postprocessor_type'] = config['postprocessor_type']

    experiment_name = f"{config['results_dir']}/{config['dataset_name']}_{config['network']}_{config['postprocessor_type']}_{config['activation_function_type']}_{config['optimizer_type']}.json"
    with open(experiment_name, 'w') as fp:
        json.dump(metrics, fp)
    return metrics

### Study 2: Resnet, CIFAR-10 (ID), CIFAR-100 (OOD)

#### Study 2(a): Adam + ReLU + Odin

In [23]:
config_cifar_adam_relu_odin = {
    "batch_size": 32,
    "n_classes": 10,
    "dataset_name": "cifar",
    "epochs": 100,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "relu",
    "network": "resnet50",
    "postprocessor_type": "odin",
    "trials": 3,
    "dataset_type": "cifar",
    "results_dir": "cifar10-study",
    "pretrained": False
} 
config_cifar_adam_relu_odin["data_loaders"] = get_data_loaders(config_cifar_adam_relu_odin)
run_full_oodn_pipeline(config_cifar_adam_relu_odin)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Running model: models/cifar_resnet50_odin_relu_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.642430305480957 time take: 0.024790525436401367
idx: 100, loss: 4.273593902587891 time take: 4.180047273635864
idx: 200, loss: 2.217350482940674 time take: 4.183010101318359
idx: 300, loss: 2.5634069442749023 time take: 4.17393159866333
idx: 400, loss: 1.8470849990844727 time take: 4.197870492935181
idx: 500, loss: 2.0578439235687256 time take: 4.173777341842651
idx: 600, loss: 2.336688756942749 time take: 4.176985025405884
idx: 700, loss: 2.0894312858581543 time take: 4.166748762130737
idx: 800, loss: 2.3092563152313232 time take: 4.266041040420532
idx: 900, loss: 1.912632942199707 time take: 4.179208040237427
idx: 1000, loss: 1.8452318906784058 time take: 4.132830619812012
idx: 1100, loss: 1.9045339822769165 time take: 4.162060737609863
idx: 1

idx: 700, loss: 1.768478274345398 time take: 4.101844549179077
idx: 800, loss: 1.6334450244903564 time take: 4.12708854675293
idx: 900, loss: 1.3116384744644165 time take: 4.131580591201782
idx: 1000, loss: 1.5481717586517334 time take: 4.133164405822754
idx: 1100, loss: 1.3579862117767334 time take: 4.1207897663116455
idx: 1200, loss: 1.616591215133667 time take: 4.107295274734497
idx: 1300, loss: 1.7291995286941528 time take: 4.1147141456604
idx: 1400, loss: 1.6823824644088745 time take: 4.239807367324829
idx: 1500, loss: 1.8291126489639282 time take: 4.107453107833862
epoch 7 time taken: 64.45949411392212s
Training epoch: 8
idx: 0, loss: 1.6590707302093506 time take: 0.023249149322509766
idx: 100, loss: 1.5848557949066162 time take: 4.166579484939575
idx: 200, loss: 1.4233311414718628 time take: 4.122305870056152
idx: 300, loss: 1.7282540798187256 time take: 4.111442565917969
idx: 400, loss: 1.3581236600875854 time take: 4.1342902183532715
idx: 500, loss: 1.4616546630859375 time tak

idx: 100, loss: 1.3700780868530273 time take: 4.119949579238892
idx: 200, loss: 1.504178524017334 time take: 4.10373854637146
idx: 300, loss: 1.520316481590271 time take: 4.099982500076294
idx: 400, loss: 1.1981079578399658 time take: 4.116159200668335
idx: 500, loss: 1.6862860918045044 time take: 4.248613357543945
idx: 600, loss: 1.7352010011672974 time take: 4.100274324417114
idx: 700, loss: 1.673231601715088 time take: 4.118092775344849
idx: 800, loss: 1.5503705739974976 time take: 4.0982019901275635
idx: 900, loss: 1.2449883222579956 time take: 4.1225714683532715
idx: 1000, loss: 1.6015021800994873 time take: 4.154203653335571
idx: 1100, loss: 1.3361696004867554 time take: 4.129671812057495
idx: 1200, loss: 1.4449882507324219 time take: 4.1100709438323975
idx: 1300, loss: 1.2082690000534058 time take: 4.126873731613159
idx: 1400, loss: 1.4492584466934204 time take: 4.1291680335998535
idx: 1500, loss: 1.9609289169311523 time take: 4.14717435836792
epoch 15 time taken: 64.51310515403

idx: 1100, loss: 1.2494267225265503 time take: 4.1029298305511475
idx: 1200, loss: 1.3682093620300293 time take: 4.25761079788208
idx: 1300, loss: 1.3230633735656738 time take: 4.1253862380981445
idx: 1400, loss: 1.3206459283828735 time take: 4.112245321273804
idx: 1500, loss: 1.7642399072647095 time take: 4.136320352554321
epoch 22 time taken: 64.5363404750824s
Training epoch: 23
idx: 0, loss: 1.8684697151184082 time take: 0.022437095642089844
idx: 100, loss: 1.2242227792739868 time take: 4.119815826416016
idx: 200, loss: 1.3838603496551514 time take: 4.147754192352295
idx: 300, loss: 1.9519283771514893 time take: 4.122519016265869
idx: 400, loss: 1.1979789733886719 time take: 4.1149842739105225
idx: 500, loss: 1.4748904705047607 time take: 4.155505418777466
idx: 600, loss: 1.507365107536316 time take: 4.148080110549927
idx: 700, loss: 1.730521559715271 time take: 4.165325880050659
idx: 800, loss: 1.337943434715271 time take: 4.131303071975708
idx: 900, loss: 1.0752066373825073 time t

idx: 400, loss: 1.1737499237060547 time take: 4.093587875366211
idx: 500, loss: 1.3312734365463257 time take: 4.0967018604278564
idx: 600, loss: 1.613966941833496 time take: 4.090072870254517
idx: 700, loss: 1.4541596174240112 time take: 4.118129014968872
idx: 800, loss: 1.4547381401062012 time take: 4.114704608917236
idx: 900, loss: 1.186826467514038 time take: 4.128931045532227
idx: 1000, loss: 1.345072627067566 time take: 4.132929801940918
idx: 1100, loss: 1.1586062908172607 time take: 4.149299383163452
idx: 1200, loss: 1.3948030471801758 time take: 4.1006176471710205
idx: 1300, loss: 1.3864014148712158 time take: 4.121099233627319
idx: 1400, loss: 1.5418606996536255 time take: 4.119306564331055
idx: 1500, loss: 1.608499526977539 time take: 4.13366174697876
epoch 30 time taken: 64.4759612083435s
Training epoch: 31
idx: 0, loss: 1.5346505641937256 time take: 0.02272200584411621
idx: 100, loss: 1.3963303565979004 time take: 4.120968818664551
idx: 200, loss: 1.3838149309158325 time tak

...

idx: 1400, loss: 1.2188926935195923 time take: 4.084561109542847
idx: 1500, loss: 1.4810242652893066 time take: 4.083250045776367
epoch 71 time taken: 64.19235730171204s
Training epoch: 72
idx: 0, loss: 1.3378510475158691 time take: 0.022359371185302734
idx: 100, loss: 1.3452297449111938 time take: 4.259092569351196
idx: 200, loss: 1.3734619617462158 time take: 4.096637487411499
idx: 300, loss: 1.5527361631393433 time take: 4.101419687271118
idx: 400, loss: 1.1521098613739014 time take: 4.100008487701416
idx: 500, loss: 1.3401960134506226 time take: 4.1183342933654785
idx: 600, loss: 1.409091591835022 time take: 4.09293794631958
idx: 700, loss: 1.5150190591812134 time take: 4.102417469024658
idx: 800, loss: 1.5238255262374878 time take: 4.07917332649231
idx: 900, loss: 1.2209625244140625 time take: 4.095714092254639
idx: 1000, loss: 1.2959414720535278 time take: 4.125247478485107
idx: 1100, loss: 1.443795084953308 time take: 4.114696979522705
idx: 1200, loss: 1.3726919889450073 time ta

idx: 700, loss: 1.519424319267273 time take: 4.125104188919067
idx: 800, loss: 1.2703531980514526 time take: 4.2646729946136475
idx: 900, loss: 1.135718822479248 time take: 4.083980083465576
idx: 1000, loss: 1.6362076997756958 time take: 4.086985111236572
idx: 1100, loss: 1.3018251657485962 time take: 4.142906665802002
idx: 1200, loss: 1.3502687215805054 time take: 4.103118181228638
idx: 1300, loss: 1.166398048400879 time take: 4.078049898147583
idx: 1400, loss: 1.2921169996261597 time take: 4.115442276000977
idx: 1500, loss: 1.9110538959503174 time take: 4.112778902053833
epoch 79 time taken: 64.3604347705841s
Training epoch: 80
idx: 0, loss: 1.333293080329895 time take: 0.02221512794494629
idx: 100, loss: 1.3626527786254883 time take: 4.093260765075684
idx: 200, loss: 1.2473862171173096 time take: 4.0919365882873535
idx: 300, loss: 1.6395701169967651 time take: 4.095915079116821
idx: 400, loss: 1.0481903553009033 time take: 4.092740535736084
idx: 500, loss: 1.4515244960784912 time ta

idx: 100, loss: 1.2760744094848633 time take: 4.09189248085022
idx: 200, loss: 1.3040345907211304 time take: 4.085116863250732
idx: 300, loss: 1.5675663948059082 time take: 4.104551076889038
idx: 400, loss: 1.4202741384506226 time take: 4.152747392654419
idx: 500, loss: 1.3985496759414673 time take: 4.092851400375366
idx: 600, loss: 1.6065247058868408 time take: 4.077431678771973
idx: 700, loss: 1.5580533742904663 time take: 4.105807781219482
idx: 800, loss: 1.4226478338241577 time take: 4.101495027542114
idx: 900, loss: 1.082322359085083 time take: 4.107697486877441
idx: 1000, loss: 1.4458897113800049 time take: 4.11318039894104
idx: 1100, loss: 1.4611766338348389 time take: 4.092067718505859
idx: 1200, loss: 1.4937890768051147 time take: 4.097015380859375
idx: 1300, loss: 1.1647557020187378 time take: 4.089705228805542
idx: 1400, loss: 1.2088645696640015 time take: 4.084718942642212
idx: 1500, loss: 1.6264084577560425 time take: 4.198615789413452
epoch 87 time taken: 64.1677229404449

idx: 1100, loss: 1.3782877922058105 time take: 4.090161323547363
idx: 1200, loss: 1.5683337450027466 time take: 4.107475996017456
idx: 1300, loss: 1.3757293224334717 time take: 4.126867771148682
idx: 1400, loss: 1.2717082500457764 time take: 4.129211664199829
idx: 1500, loss: 1.7574912309646606 time take: 4.089621305465698
epoch 94 time taken: 64.16421890258789s
Training epoch: 95
idx: 0, loss: 1.4611996412277222 time take: 0.024432897567749023
idx: 100, loss: 1.3799128532409668 time take: 4.109936952590942
idx: 200, loss: 1.3747881650924683 time take: 4.11156439781189
idx: 300, loss: 1.655493974685669 time take: 4.103376626968384
idx: 400, loss: 1.190886378288269 time take: 4.081382513046265
idx: 500, loss: 1.3663756847381592 time take: 4.107540845870972
idx: 600, loss: 1.549180507659912 time take: 4.14205265045166
idx: 700, loss: 1.5422576665878296 time take: 4.1755454540252686
idx: 800, loss: 1.540257453918457 time take: 4.105756521224976
idx: 900, loss: 0.9602973461151123 time take

{0: {'dataset': 'cifar',
  'FPR@95': '92.51',
  'AUROC': '56.12',
  'AUPR_IN': '56.08',
  'AUPR_OUT': '54.96',
  'CCR_4': '0.03',
  'CCR_3': '0.42',
  'CCR_2': '2.08',
  'CCR_1': '10.78',
  'ACC': '41.22',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'},
 1: {'dataset': 'cifar',
  'FPR@95': '91.57',
  'AUROC': '60.16',
  'AUPR_IN': '61.22',
  'AUPR_OUT': '57.54',
  'CCR_4': '0.20',
  'CCR_3': '0.60',
  'CCR_2': '3.31',
  'CCR_1': '14.30',
  'ACC': '43.02',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'},
 2: {'dataset': 'cifar',
  'FPR@95': '91.37',
  'AUROC': '62.52',
  'AUPR_IN': '62.89',
  'AUPR_OUT': '59.61',
  'CCR_4': '0.05',
  'CCR_3': '0.85',
  'CCR_2': '4.29',
  'CCR_1': '15.80',
  'ACC': '47.17',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'}}

#### Study 2 (b.) Adam + Softplus + odin

In [ ]:
config_cifar_adam_softplus_odin = {
    "batch_size": 32,
    "n_classes": 10,
    "dataset_name": "cifar",
    "epochs": 100,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "softplus",
    "network": "resnet50",
    "postprocessor_type": "odin",
    "trials": 3,
    "dataset_type": "cifar",
    "results_dir": "cifar10-study",
    "pretrained": False
}
config_cifar_adam_softplus_odin["data_loaders"] = get_data_loaders(config_cifar_adam_softplus_odin)
run_full_oodn_pipeline(config_cifar_adam_softplus_odin)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Running model: models/cifar_resnet50_odin_softplus_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.9400904178619385 time take: 0.022324323654174805
idx: 100, loss: 5.485348701477051 time take: 4.096525430679321
idx: 200, loss: 5.1429057121276855 time take: 4.111398935317993
idx: 300, loss: 3.4952378273010254 time take: 4.126551628112793
idx: 400, loss: 3.251206159591675 time take: 4.083424806594849
idx: 500, loss: 3.5177512168884277 time take: 4.073462724685669
idx: 600, loss: 3.0264832973480225 time take: 4.1066319942474365
idx: 700, loss: 3.4363059997558594 time take: 4.0969626903533936
idx: 800, loss: 2.5099704265594482 time take: 4.234477519989014
idx: 900, loss: 2.5756146907806396 time take: 4.117691516876221
idx: 1000, loss: 2.4096317291259766 time take: 4.1335954666137695
idx: 1100, loss: 2.5305287837982178 time take: 4.09361171722

idx: 700, loss: 2.8071746826171875 time take: 4.108215808868408
idx: 800, loss: 2.7940328121185303 time take: 4.104716062545776
idx: 900, loss: 2.247343063354492 time take: 4.099836111068726
idx: 1000, loss: 1.9820953607559204 time take: 4.095187425613403
idx: 1100, loss: 2.071901798248291 time take: 4.101616859436035
idx: 1200, loss: 1.7930326461791992 time take: 4.12448787689209
idx: 1300, loss: 2.8186168670654297 time take: 4.143683671951294
idx: 1400, loss: 2.2102015018463135 time take: 4.1360859870910645
idx: 1500, loss: 2.989084243774414 time take: 4.23716402053833
epoch 7 time taken: 64.39987754821777s
Training epoch: 8
idx: 0, loss: 2.5929460525512695 time take: 0.022413253784179688
idx: 100, loss: 2.3263602256774902 time take: 4.107846021652222
idx: 200, loss: 1.8463211059570312 time take: 4.108522653579712
idx: 300, loss: 3.3185489177703857 time take: 4.12127947807312
idx: 400, loss: 2.661604881286621 time take: 4.150969505310059
idx: 500, loss: 3.2336692810058594 time take: 

idx: 100, loss: 2.3893938064575195 time take: 4.119708061218262
idx: 200, loss: 2.375619411468506 time take: 4.13898229598999
idx: 300, loss: 2.826690435409546 time take: 4.105683326721191
idx: 400, loss: 2.2055325508117676 time take: 4.134119749069214
idx: 500, loss: 2.241858959197998 time take: 4.130999803543091
idx: 600, loss: 2.884021759033203 time take: 4.235384464263916
idx: 700, loss: 3.8363029956817627 time take: 4.084251403808594
idx: 800, loss: 2.290513515472412 time take: 4.100091218948364
idx: 900, loss: 2.2360012531280518 time take: 4.10809326171875
idx: 1000, loss: 2.456415891647339 time take: 4.1208178997039795
idx: 1100, loss: 2.9646921157836914 time take: 4.1170814037323
idx: 1200, loss: 2.350405693054199 time take: 4.144125699996948
idx: 1300, loss: 2.3812310695648193 time take: 4.128928184509277
idx: 1400, loss: 1.8794490098953247 time take: 4.111865282058716
idx: 1500, loss: 2.818408727645874 time take: 4.113945484161377
epoch 15 time taken: 64.50151300430298s
Train

idx: 1100, loss: 3.1001486778259277 time take: 4.107495307922363
idx: 1200, loss: 1.9159338474273682 time take: 4.121116876602173
idx: 1300, loss: 2.230834484100342 time take: 4.281702518463135
idx: 1400, loss: 2.03924298286438 time take: 4.121699094772339
idx: 1500, loss: 2.6470682621002197 time take: 4.09529447555542
epoch 22 time taken: 64.57038569450378s
Training epoch: 23
idx: 0, loss: 2.8895890712738037 time take: 0.022577524185180664
idx: 100, loss: 1.872208595275879 time take: 4.095221519470215
idx: 200, loss: 2.1307146549224854 time take: 4.101665258407593
idx: 300, loss: 3.1175224781036377 time take: 4.108230113983154
idx: 400, loss: 1.7871814966201782 time take: 4.125839710235596
idx: 500, loss: 3.200544834136963 time take: 4.150827169418335
idx: 600, loss: 2.0987298488616943 time take: 4.119591236114502
idx: 700, loss: 4.107021331787109 time take: 4.1043925285339355
idx: 800, loss: 2.118429660797119 time take: 4.124672174453735
idx: 900, loss: 2.266782522201538 time take: 4

idx: 400, loss: 1.880631923675537 time take: 4.089143753051758
idx: 500, loss: 2.5125842094421387 time take: 4.2229673862457275
idx: 600, loss: 2.9672224521636963 time take: 4.093381881713867
idx: 700, loss: 3.4907662868499756 time take: 4.1004908084869385
idx: 800, loss: 1.7094521522521973 time take: 4.1288909912109375
idx: 900, loss: 1.9909257888793945 time take: 4.107529163360596
idx: 1000, loss: 2.6961286067962646 time take: 4.112977504730225
idx: 1100, loss: 1.714165449142456 time take: 4.09544825553894
idx: 1200, loss: 2.0552055835723877 time take: 4.104825735092163
idx: 1300, loss: 2.3521339893341064 time take: 4.114988565444946
idx: 1400, loss: 1.9739277362823486 time take: 4.146896123886108
idx: 1500, loss: 2.1803512573242188 time take: 4.115297079086304
epoch 30 time taken: 64.35524415969849s
Training epoch: 31
idx: 0, loss: 2.738518476486206 time take: 0.022493362426757812
idx: 100, loss: 2.952077865600586 time take: 4.128525972366333
idx: 200, loss: 1.952338695526123 time t

...

idx: 400, loss: 3.043217897415161 time take: 4.109385251998901
idx: 500, loss: 2.217162609100342 time take: 4.104505300521851
idx: 600, loss: 2.794947862625122 time take: 4.257663011550903
idx: 700, loss: 3.356848955154419 time take: 4.1457109451293945
idx: 800, loss: 1.8626168966293335 time take: 4.120071649551392
idx: 900, loss: 2.5152976512908936 time take: 4.171619892120361
idx: 1000, loss: 2.3391878604888916 time take: 4.144917011260986
idx: 1100, loss: 1.9030312299728394 time take: 4.1293275356292725
idx: 1200, loss: 2.2430484294891357 time take: 4.130168914794922
idx: 1300, loss: 2.3559155464172363 time take: 4.139855861663818
idx: 1400, loss: 2.19649076461792 time take: 4.131213903427124
idx: 1500, loss: 2.692152976989746 time take: 4.144739866256714
epoch 68 time taken: 64.68627142906189s
Training epoch: 69
idx: 0, loss: 2.2218611240386963 time take: 0.022417306900024414
idx: 100, loss: 1.898298740386963 time take: 4.1468424797058105
idx: 200, loss: 2.113057851791382 time take

idx: 1400, loss: 1.990929365158081 time take: 4.138254880905151
idx: 1500, loss: 2.631664514541626 time take: 4.112443685531616
epoch 75 time taken: 64.76096820831299s
Training epoch: 76
idx: 0, loss: 2.102125883102417 time take: 0.022577762603759766
idx: 100, loss: 2.7370262145996094 time take: 4.1522133350372314
idx: 200, loss: 2.134782314300537 time take: 4.119861602783203
idx: 300, loss: 3.6552302837371826 time take: 4.121166467666626
idx: 400, loss: 2.3762624263763428 time take: 4.10661506652832
idx: 500, loss: 2.9749948978424072 time take: 4.132277488708496
idx: 600, loss: 3.044435977935791 time take: 4.116989850997925
idx: 700, loss: 2.469808578491211 time take: 4.119347333908081
idx: 800, loss: 1.6381781101226807 time take: 4.084413766860962
idx: 900, loss: 1.7733861207962036 time take: 4.100072145462036
idx: 1000, loss: 1.718733787536621 time take: 4.107823371887207
idx: 1100, loss: 1.7473633289337158 time take: 4.209038257598877
idx: 1200, loss: 2.595149278640747 time take: 4

idx: 700, loss: 2.612661600112915 time take: 4.091521501541138
idx: 800, loss: 2.0829434394836426 time take: 4.099865913391113
idx: 900, loss: 2.515453577041626 time take: 4.094320774078369
idx: 1000, loss: 2.3880455493927 time take: 4.1323418617248535
idx: 1100, loss: 1.693222999572754 time take: 4.10141134262085
idx: 1200, loss: 1.9837976694107056 time take: 4.083273887634277
idx: 1300, loss: 3.0197913646698 time take: 4.1062328815460205
idx: 1400, loss: 1.9967581033706665 time take: 4.102694988250732
idx: 1500, loss: 3.1266820430755615 time take: 4.109030485153198
epoch 83 time taken: 64.18324422836304s
Training epoch: 84
idx: 0, loss: 2.697432041168213 time take: 0.022053956985473633
idx: 100, loss: 1.75412917137146 time take: 4.078891038894653
idx: 200, loss: 2.5410826206207275 time take: 4.085751533508301
idx: 300, loss: 2.853717565536499 time take: 4.230431079864502
idx: 400, loss: 2.5781524181365967 time take: 4.1088902950286865
idx: 500, loss: 3.136338949203491 time take: 4.08

idx: 100, loss: 2.1674561500549316 time take: 4.09288215637207
idx: 200, loss: 2.438462495803833 time take: 4.101112604141235
idx: 300, loss: 2.837115526199341 time take: 4.105496883392334
idx: 400, loss: 1.471131443977356 time take: 4.172549724578857
idx: 500, loss: 1.8333771228790283 time take: 4.083679437637329
idx: 600, loss: 2.7564666271209717 time take: 4.098332166671753
idx: 700, loss: 4.265981674194336 time take: 4.101928472518921
idx: 800, loss: 1.6334683895111084 time take: 4.103339910507202
idx: 900, loss: 1.4866218566894531 time take: 4.120863676071167
idx: 1000, loss: 2.228166341781616 time take: 4.10866379737854
idx: 1100, loss: 2.2818140983581543 time take: 4.200334072113037
idx: 1200, loss: 1.6838973760604858 time take: 4.079050302505493
idx: 1300, loss: 2.7004942893981934 time take: 4.087135553359985
idx: 1400, loss: 1.6852507591247559 time take: 4.090198755264282
idx: 1500, loss: 1.8014037609100342 time take: 4.085691213607788
epoch 91 time taken: 64.22058749198914s
T

idx: 1100, loss: 2.0081732273101807 time take: 4.120847463607788
idx: 1200, loss: 2.369462251663208 time take: 4.113290071487427
idx: 1300, loss: 2.4151012897491455 time take: 4.165461301803589
idx: 1400, loss: 1.9285134077072144 time take: 4.110636472702026
idx: 1500, loss: 2.796034574508667 time take: 4.118157863616943
epoch 98 time taken: 64.34691333770752s
Training epoch: 99
idx: 0, loss: 2.399913787841797 time take: 0.022266626358032227
idx: 100, loss: 1.7870452404022217 time take: 4.098530530929565
idx: 200, loss: 2.4170544147491455 time take: 4.2479493618011475
idx: 300, loss: 3.19677996635437 time take: 4.099990367889404
idx: 400, loss: 1.5875859260559082 time take: 4.104693651199341
idx: 500, loss: 2.571455717086792 time take: 4.13665246963501
idx: 600, loss: 2.8675878047943115 time take: 4.118646144866943
idx: 700, loss: 3.0254251956939697 time take: 4.092961072921753
idx: 800, loss: 1.8165124654769897 time take: 4.094493389129639
idx: 900, loss: 2.3485748767852783 time take: